In [5]:
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score    #0.19.1
from scipy.stats import pearsonr
from mxnet.gluon.model_zoo import vision
import cv2           #3.4.1'
import copy
import mxnet as mx   #1.2.0
import numpy as np   #1.14.5


In [6]:
mnist_train = mx.gluon.data.vision.datasets.MNIST(train=True)
mnist_test = mx.gluon.data.vision.datasets.MNIST(train=False)

In [7]:
X = np.zeros((7000, 28, 28))
for i, (data, label) in enumerate(mnist_train):
    if i >=6000:
        break
    X[i] = data.asnumpy()[:,:,0]
for i, (data, label) in enumerate(mnist_test):
    if i >=1000:
        break
    X[6000+i] = data.asnumpy()[:,:,0]

In [8]:
#Use a seed so that we get the same random permutation each time
np.random.seed(1)
p = np.random.permutation(X.shape[0])
X = X[p]

In [9]:
X = np.asarray([cv2.resize(x, (64,64)) for x in X])

In [10]:
X = X.astype(np.float32, copy=False)/(255.0/2) - 1.0

In [11]:
X = X.reshape((7000, 1, 64, 64))
X = np.tile(X, (1, 3, 1, 1))

In [12]:
batch_size = 25
image_iter = mx.io.NDArrayIter(X, batch_size=batch_size)

In [13]:
class RandIter(mx.io.DataIter):
    def __init__(self, batch_size, ndim):
        self.batch_size = batch_size
        self.ndim = ndim
        self.provide_data = [('rand', (batch_size, ndim, 1, 1))]
        self.provide_label = []

    def iter_next(self):
        return True

    def getdata(self):
        #Returns random numbers from a gaussian (normal) distribution
        #with mean=0 and standard deviation = 1
        return [mx.random.normal(0, 1.0, shape=(self.batch_size, self.ndim, 1, 1))]

In [14]:
Z = 48 #100 512 #2048     #This is your latent space size
rand_iter = RandIter(batch_size, Z)